# Tutoriel d'exemple de résolution d'un problème de machine learning à l'aide de la régression linéaire

## Problème à résoudre

Notre travail va consister à créer un algorithme qui prédit le prix médian des maisons dans un quartier de Californie en fonction de données telles que les revenus médians des habitants du quartier, la population du quartier, le nombre médian de chambres et de pièces... Pour cela, nous utiliserons [ce set de données](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html).

## Techniques et outils utilisés

Pour résoudre ce problème, nous allons faire appel à une technique **d'apprentissage supervisé** basée sur la **régression linéaire**. Voici une description de ces termes :
- L'apprentissage supervisé est une technique qui consiste à entraîner un modèle de prédiction en lui donnant des sets de données. Dans notre cas, nous allons utiliser nos données pour apprendre à notre modèle à prédire le prix médian des maisons dans un quartier selon les caractéristiques de ce dernier.
- La régression linéaire consiste à trouver une approximation de la fonction qui définit la relation entre différentes valeurs.

Nous allons utiliser la librairie scikit-learn pour réaliser ce projet.

## Code

### Importations des librairies

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

### Importation du dataset

Commençons par importer et explorer nos données :

In [10]:
california_housing = fetch_california_housing(as_frame=True)
print(california_housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Comme cette description nous l'indique, ce dataset contient 20640 lignes, où chaque ligne correspond à un quartier. La valeur "cible", celle que nous cherchons à prédire, est la valeur médiane des maisons pour un quartier en centaines de milliers de dollars. Les 8 attributs numériques mentionnés sont les valeurs à partir desquelles on pourra réaliser des prédictions. Affichons les données pour voir plus clairement le format du dataset :

In [11]:
print(california_housing.frame)

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0      8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1      8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2      7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3      5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4      3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   
...       ...       ...       ...        ...         ...       ...       ...   
20635  1.5603      25.0  5.045455   1.133333       845.0  2.560606     39.48   
20636  2.5568      18.0  6.114035   1.315789       356.0  3.122807     39.49   
20637  1.7000      17.0  5.205543   1.120092      1007.0  2.325635     39.43   
20638  1.8672      18.0  5.329513   1.171920       741.0  2.123209     39.43   
20639  2.3886      16.0  5.254717   1.162264      1387.0  2.616981     39.37   

       Longitude  MedHouseVal  
0      

### Phase d'apprentissage du modèle

La première étape lors de l'utilisation d'une méthode d'apprentissage supervisée, c'est l'apprentissage. Dans cette étape, nous allons construire notre modèle en lui donnant un certain nombre d'exemples depuis notre base de données. La règle de base lors de cette étape est de séparer la base de données en deux : une partie pour l'apprentissage et une partie pour le test. Ceci afin de ne pas montrer lors de la phase de test des exemples de chiffres qui auraient servis lors de la phase d'apprentissage, car alors le test serait biaisé.

In [29]:
housing_df = california_housing.frame
# Split data with 70% to train and 30% to test
train, test = train_test_split(housing_df, test_size=0.3)
x_train = train.drop(columns=["MedHouseVal"])
y_train = train["MedHouseVal"]
x_test = test.drop(columns=["MedHouseVal"])
y_test = test["MedHouseVal"]

        MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
18935   7.2589      21.0  8.035714   1.107143       812.0  2.636364     38.23   
2701    3.2396      33.0  5.460000   1.076000       825.0  3.300000     32.85   
511    13.4990      42.0  8.928358   1.000000      1018.0  3.038806     37.82   
16129   4.1339      52.0  4.567059   1.025882       955.0  2.247059     37.78   
18565   2.4461      17.0  4.903704   1.090741      1193.0  2.209259     36.92   
...        ...       ...       ...        ...         ...       ...       ...   
938     4.1406       6.0  4.566510   1.051643      1911.0  2.990610     37.54   
13880   3.1505       9.0  5.623120   1.054511      3515.0  3.303571     34.46   
3444    2.6429      33.0  4.494565   1.043478       981.0  5.331522     34.25   
5993    3.2566      36.0  4.926421   1.096990       785.0  2.625418     34.09   
13607   2.1465      22.0  4.430486   1.103294      4523.0  2.525405     34.15   

       Longitude  
18935   

Puis nous mettons en place notre modèle, l'entraînons et visualisons les résultats de l'entraînement :

In [31]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(x_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(x_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [ 4.30379030e-01  9.87527398e-03 -9.61014273e-02  5.78267170e-01
 -3.69681847e-06 -3.84644166e-03 -4.19689737e-01 -4.34307398e-01]
Mean squared error: 0.52
Coefficient of determination: 0.61


Ce script affiche plusieurs informations importantes :
- Les coefficients : la régression linéaire consiste à trouver une fonction qui permet de prédire le prix médian des maisons à partir de 8 valeurs. Cette fonction est de la forme f(x) = a*f1 + b*f2 ... + h*f8 + i, où f1, f2, etc... sont les valeurs données en entrée et a, b, c etc.. sont les coefficients qui multiplient chaque valeur. Il s'agit des coefficients que l'on affiche ici.
- La MSE (mean squared error) est un indicateur qui évalue la précision de notre modèle.
- Le coefficient de détermination est un indicateur allant de 0 à 1 qui évalue la précision du modèle. Le coefficient de 65 % que l'on obtient veut dire que 65 % des variations observées dans le prix médian des maisons peuvent être expliquées par les 8 variables d'entrée.

## Conclusion

Ce tutoriel est maintenant terminé ! N'hésitez pas à tester la régression linéaire sur différents datasets. Vous pouvez aussi essayer les autres formes de régression proposées par scikit-learn.